In [ ]:
import os, glob
import pandas as pd
import pyranges as pr
from utils.ref import get_tss_pyranges

In [4]:
pth = os.path.join('..', '..', '..', 'data')
pth_in = os.path.join(pth, 'unzip', 'GSE230319')
pth_ref = os.path.join(pth, 'ref')
pth_mod = os.path.join(pth, 'features', 'biomart', 'modules')
pth_out = os.path.join(pth, 'DELAY', 'rRNAModifications')

In [24]:
df = pd.read_csv(os.path.join(pth_mod, 'union.csv'))
df

,mmusculus,hsapiens
0,1700017B05Rik,C15orf39
1,A2m,A2M
2,Actn1,ACTN1
3,Adora2b,ADORA2B
4,Agtrap,AGTRAP
...,...,...
472,Zfp503,ZNF503
473,Zic2,ZIC2
474,Zic3,ZIC3
475,Zic5,ZIC5


In [3]:
tss = get_tss_pyranges(os.path.join(pth_ref, 'hg38.refGene.gtf.gz'))
tss

/home/caleb/MesenCoder/DELAY/preprocessing/ground-truth/utils/ref.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(get_upstream_tss))


,Chromosome,Start,End,Strand,gene_id
0,chr1,12716109,12716110,+,AADACL3
1,chr1,12644546,12644547,+,AADACL4
2,chr1,94418376,94418377,+,ABCD3
3,chr1,75724346,75724347,+,ACADM
4,chr1,226148002,226148003,+,ACBD3-AS1
...,...,...,...,...,...
26133,chrY,8683877,8683878,-,TTTY18
26134,chrY,9334831,9334832,-,TTTY20
26135,chrY,22298875,22298876,-,TTTY5
26136,chrY,13480669,13480670,-,UTY


In [ ]:
peaks_dict = {
    '*TWIST1_*' : 'Twist1',
    '*ALX1*'    : 'Alx1',
    '*ALX4*'    : 'Alx4'}

g1_list, g2_list = list(), list()
for regex, g1 in peaks_dict.items():
    for fn in sorted(glob.glob(os.path.join(pth_in, regex))):
        print(g1, fn)

        g2 = (pr.read_bed(fn)
              .nearest(tss, apply_strand_suffix = True)
              .df.gene_id.unique())
        
        g1_list.extend([g1] * len(g2))
        g2_list.extend(g2)
        
refNet = pd.DataFrame({'Gene1' : g1_list, 'Gene2' : g2_list})
refNet.drop_duplicates(['Gene1', 'Gene2'], inplace = True)
refNet = refNet.loc[refNet.Gene2.isin(df.hsapiens)]
gmap = df.set_index('hsapiens').mmusculus.to_dict()
refNet['Gene2'] = refNet.Gene2.map(gmap)

refNet.groupby('Gene1').size() / df.shape[0]


Twist1 ../../../data/unzip/GSE230319/GSM7213613_TWIST1FV_dTAG-0h_r1_TWIST1_peaks.narrowPeak.gz
Twist1 ../../../data/unzip/GSE230319/GSM7213615_TWIST1FV_dTAG-0h_r2_TWIST1_peaks.narrowPeak.gz
Alx1 ../../../data/unzip/GSE230319/GSM7213651_ALX1FV_dTAG-0h_r1_V5_peaks.narrowPeak.gz
Alx1 ../../../data/unzip/GSE230319/GSM7213653_ALX1FV_dTAG-0h_r2_V5_peaks.narrowPeak.gz
Alx4 ../../../data/unzip/GSE230319/GSM7213748_TWIST1FV_dTAG-0h_CnR_ALX4_peaks.narrowPeak.gz
Gene1
Alx1      0.092243
Alx4      0.211740
Twist1    0.672956
dtype: float64


,Gene1,Gene2
2,Twist1,Atad3a
33,Twist1,Eva1b
62,Twist1,Vcam1
82,Twist1,Crabp2
85,Twist1,Ddr2
...,...,...
25749,Alx4,Sulf2
25754,Alx4,Snai1
25797,Alx4,Psmg1
25871,Alx4,Cited1
